In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
random.seed(42)

from PIL import Image

In [2]:
key = 'oil'
train_images = {0:[], 1:[], 2:[], 3:[]}                                     
train_jsons = {0:[], 1:[], 2:[], 3:[]}
train_path = './dataset/train' 
with open(os.path.join(train_path, "annotations.json"), "r") as json_file:
    train_ann = json.load(json_file)
images_info = train_ann['images']

for image_info in images_info:
    image_name = image_info['file_name']                                
    file_name = image_name.replace('jpg', 'json')                       

    with open(os.path.join(train_path, file_name), "r") as json_file:   
        img_json = json.load(json_file)             
    label = {}
    label[key] = img_json[key]
    if label[key] == -1: continue
    
    image_path = os.path.join(train_path, image_name)                   
    image = cv2.imread(image_path)                                      
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    part = img_json['part']
    train_images[part].append(image)
    train_jsons[part].append(label)

In [3]:
combined_images = {0: [], 1: [], 2: [], 3: [], 4: []}

aim = 5000

forehead_num = len(train_images[1])-1
nose_num = len(train_images[2])-1
lip_num = len(train_images[3])-1

num = 0
while num < aim:
    forehead_idx = random.randint(0, forehead_num)
    nose_idx = random.randint(0, nose_num)
    lip_idx = random.randint(0, lip_num)
    label = round((train_jsons[1][forehead_idx][key]+train_jsons[2][nose_idx][key]) / 2)
    
    if len(combined_images[label]) >= 1000: continue
    combined_image = np.concatenate((train_images[1][forehead_idx], train_images[2][nose_idx], train_images[3][lip_idx]), axis = 0)
    combined_images[label].append(combined_image)
    num += 1

In [ ]:
labels_num = []

for i in range(5):
    labels_num.append(len(combined_images[i]))

plt.bar(np.arange(5), labels_num)

In [5]:
train_path = './dataset/train'
save_path = os.path.join(train_path, f'combined_random_{key}_images')
if not os.path.exists(os.path.join(save_path)):
    os.mkdir(save_path)

image_num = 0

for label, images in combined_images.items():
    json_content = {key: label}
    for image in images:
        img_name = str(image_num).zfill(4)
        img = Image.fromarray(image)
        img.save(os.path.join(save_path, img_name+'.jpg'))
        with open(os.path.join(save_path, img_name+'.json'), 'w') as json_file:
            json.dump(json_content, json_file)
        image_num += 1